# Artifact Hub Helm charts security posture

Data has been generated using https://github.com/vicenteherrera/psa-checker

In [1]:
import yaml
import os
from string import ascii_lowercase

We load the data from disk

In [2]:
charts_pss = yaml.safe_load(open("helm_charts_pss.yaml", 'r'))

Count status for all charts

In [3]:
i=0
count = {
    "total":0,
    "privileged":0,
    "baseline":0,
    "restricted":0,
    "error_download":0,
    "error_template":0,
}
brbuckets = {}
brzero = 0
br_min_buckets = -700
br_size_buckets = -50
br_n_buckets = int (br_min_buckets / br_size_buckets)
brmin = br_non_evaluable = br_no_workload = 0
for i in range(0, br_min_buckets, br_size_buckets):
    brbuckets[str(i)] = 0

calpha = {}
for l in ascii_lowercase:
    calpha[l]=0

print("# Counting")
date = ""
keys_pss = charts_pss.keys()    
for key in keys_pss:
    dic_chart = charts_pss[key]
    level = "unknown"
    i += 1
    calpha[dic_chart["repository"]["name"][0]] += 1
    if "pss" in dic_chart:
        if dic_chart["pss"]["date"] > date:
            date = dic_chart["pss"]["date"]
        level = dic_chart["pss"]["level"]

        if "badrobot" in dic_chart:
            score = dic_chart["badrobot"]["score"]
            if score == "":
                br_non_evaluable += 1
            elif dic_chart["pss"]["level"] in ["empty_no_object", "no_pod_object_but_crd", "no_pod_object"]:
                br_no_workload += 1
            else :
                score = int(score)
                if score < brmin: brmin = int(score)
                if score == 0:
                    brzero += 1
                else:
                    for i in range(0, br_min_buckets + br_size_buckets, br_size_buckets):
                        if i >= score and score > ( i + br_size_buckets ):
                            brbuckets[str(i)] += 1
                            break

    count["total"] +=1
    if level not in count:
        count[level] = 1
    else:
        count[level] += 1

# Counting


In [4]:
count

{'total': 10194,
 'privileged': 549,
 'baseline': 4320,
 'restricted': 32,
 'error_download': 2265,
 'error_template': 757,
 'empty_no_object': 376,
 'no_pod_object_but_crd': 1247,
 'version_not_evaluable': 491,
 'no_pod_object': 157}

In [5]:
brzero

832

In [6]:
brbuckets

{'0': 3291,
 '-50': 205,
 '-100': 63,
 '-150': 15,
 '-200': 9,
 '-250': 5,
 '-300': 2,
 '-350': 1,
 '-400': 1,
 '-450': 0,
 '-500': 0,
 '-550': 0,
 '-600': 0,
 '-650': 1}